In [1]:
import requests
import re
import pandas as pd
import urllib.parse
import codecs
import time
from random import randint
from pyquery import PyQuery as pq

In [2]:
session_requests = requests.session()
auth_token = None
    
login_url = 'https://danbooru.donmai.us/session/new'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
result = session_requests.get(login_url, headers = headers)

login_page = pq(result.text)
authenticity_token = urllib.parse.quote(login_page('input').attr('value'),safe='')
#print(authenticity_token)

login_data = {
    'authenticity_token': authenticity_token,
    'url': '',
    'name': 'komeiji_nori',
    'password': ,
    'remember': '1',   
    'commit': 'Submit'
}

resp = session_requests.post(url = 'https://danbooru.donmai.us/session', data = login_data, headers = headers)

In [3]:
dict_file = codecs.open('kantai_chara_dict_submarine_fix.txt', 'r', 'utf-8')
chara_dict = {}

for line in dict_file:
    name_pair = (line[:-1]).split('\t')
    chara_dict[name_pair[0]] = name_pair[1]

chara_dict

{'长10炮酱': 'chou-10cm-hou-chan',
 '照月的长10炮酱': 'chou-10cm-hou-chan_%28teruzuki%27s%29',
 '初雪的长10炮酱': 'chou-10cm-hou-chan_%28hatsuzuki%27s%29',
 'Z1': 'z1_leberecht_maass_%28kantai_collection%29',
 'Z3': 'z3_max_schultz_%28kantai_collection%29',
 '伊8 ("Hachi")': 'i-8_%28kantai_collection%29',
 '伊19 ("Iku")': 'i-19_%28kantai_collection%29',
 '伊26 ("Nimu")': 'i-26_%28kantai_collection%29',
 '伊58 ("Gouya")': 'i-58_%28kantai_collection%29',
 '伊168 ("Imuya")': 'i-168_%28kantai_collection%29',
 '伊400 ("Shion")': 'i-400_%28kantai_collection%29',
 '伊401 ("Shioi")': 'i-401_%28kantai_collection%29',
 'U-511 ("Yuu")': 'u-511_%28kantai_collection%29',
 '呂500 ("Ro")': 'ro-500_%28kantai_collection%29',
 '伊13 ("Hitomi")': 'i-13_%28kantai_collection%29',
 '伊14 ("Iyo")': 'i-14_%28kantai_collection%29',
 'UIT-25 ("Ui")': 'uit-25_%28kantai_collection%29',
 '伊504 ("Go")': 'i-504_%28kantai_collection%29'}

In [4]:
def count_danbooru(key):
    
    output_dict = {}
    parent_flag = ['all','parent%3Anone']
    rating_flag = ['all','rating%3Aexplicit','rating%3Aquestionable','rating%3Asafe']
    
    for i in parent_flag:
        for j in rating_flag:
            
            #build query url
            chara_url = 'https://danbooru.donmai.us/counts/posts?tags=+-official_art'
            if i != 'all':
                chara_url = chara_url + '+' + i
            if j != 'all':
                chara_url = chara_url + '+' + j
            if key != '島風':
                chara_url = chara_url + '+' + chara_dict[key]
            else:
                chara_url = chara_url + '+' + chara_dict[key] + '+-shimakaze_%28kantai_collection%29_%28cosplay%29'

            #get query url and count
            chara_count_page = session_requests.get(chara_url)
            chara_count_query = pq(chara_count_page.text)

            for tag_div in chara_count_query('div'):
                div_query = pq(tag_div)
                div_id = div_query.attr('id')
                if div_id == 'a-posts':
                    count = div_query.remove('a').text()
                    count = re.findall(r'\d+', count)[0]

            output_dict_key = (key, urllib.parse.unquote(i), urllib.parse.unquote(j))
            output_dict[output_dict_key] = count
    
            # Waits before sending next request
            time.sleep(1 + randint(0,10))
            
    return output_dict

In [ ]:
final_result = {}

for i in chara_dict:
    a = count_danbooru(i)
    final_result = {**final_result, **a}

final_output_file = codecs.open('fixing_final.txt', 'w', 'utf-8')

for i in final_result:
    chara_name, parent_flag, ratings_flag = i
    final_output_file.write('{}\t{}\t{}\t{}\n'.format(chara_name, parent_flag, ratings_flag, final_result[i]))

final_output_file.close()